In [1]:
import pandas as pd
import glob
import os
import seaborn as sns
sns.set()

In [2]:
debug = False
folder = os.path.join("..", "..","results_all", "micro_history_opti_vm")
outputFile = "../../results_all/micro_history_opti_vm.csv"

In [3]:
def load_microbench_csv(path: str) -> pd.DataFrame:
    """load csv and do some basic preprocess(some benchmark names are broken)
    """

    df: pd.DataFrame = pd.read_csv(path,header=None,
                        names=[
                            "suiteRun",
                            "baseline",
                            "path",
                            "name",
                            "version",
                            "invocations",
                            "runtime"],
                    sep=";")
    

    df = df[['name', 'suiteRun', 'path', 'version','runtime','invocations']]

    df.sort_values(by=["name"], inplace=True)


    df.loc[(df["name"] == "16") & (df["runtime"] < 2500), "name"] = "BenchmarkStorageAddRows1"
    df.loc[(df["name"] == "16") & (df["runtime"] < 10000), "name"] = "BenchmarkStorageAddRows2"
    df.loc[(df["name"] == "16") & (df["runtime"] < 100000), "name"] = "BenchmarkStorageAddRows3"
    df.loc[(df["name"] == "16"), "name"] = "BenchmarkStorageAddRows4"

    

    
    df = df.assign(suiteRun=df['suiteRun'].str.extract(r"(?P<suiteRun>\d)-[\d]-[\d]", expand=False))
    df = df.assign(name=df['name'].str.strip('\"'))
    
    df['version'] = df['version'].apply(pd.to_numeric)
    df['suiteRun'] = df['suiteRun'].apply(pd.to_numeric)
    
    return df


def aggregate_data(df: pd.DataFrame) -> list:
    group: pd.DataFrame
    instanceRun = df['instanceRun'].sample().values
    errorCounter = [[0 for x in range(3)] for y in range(2)]
    

    aggr = []
    groupBy: pd.core.groupby.DataFrameGroupBy = df.groupby(["suiteRun", "path", "name"])
    for names, group in groupBy:
        if debug:
            print(f" Aggregating values for benchmark benchmark{names[2]} with path{names[1]} in run {names[0]}")     

        df1 = group[group['version'] == 1]
        df2 = group[group['version'] == 2]
        #There must be 5 values in each dataframe (5 iterations)
        error = False
        if not len(df1) == 5:
            print(f"  There are {len(df1)} elements for version 1, "
            f"experimentRun: {instanceRun}, suiteRun: {names[0]}, commitNumber: {commitNumber}, "
            f"path: {names[1]}, name: {names[2]}")
            error = True
            errorCounter[0][int(names[0])] += 1
        if not len(df2) == 5:
            print(f"  There are {len(df2)} elements for version 2, "
            f"experimentRun: {instanceRun}, suiteRun: {names[0]}, commitNumber: {commitNumber}, "
            f"path: {names[1]}, name: {names[2]}")
            error = True
            errorCounter[1][int(names[0])] += 1
        #Find median
        if not error:
            aggr.append(group)

    print(f"  Found {len(groupBy)} benchmarks")
    print(f"  Errorcounter ([version1], [version2]): {errorCounter}")


    return aggr

In [4]:
aggr = []

all_runs = glob.glob(os.path.join(folder, "run[0-9]"))
for fullRunFolder in all_runs:
    all_numbers = glob.glob(os.path.join(fullRunFolder, "*"))
    for fullNumberFolder in all_numbers:

        all_files = glob.glob(os.path.join(fullNumberFolder, "*"))
        size = len(all_files)

        if size < 7:
            print(f"only {size} elements in folder {fullNumberFolder}, skip.")
        else:
            tmp = fullNumberFolder.split(os.sep)

            #Parse run and commit number from path structure
            instanceRun = tmp[-2][-1]
            commitNumber = tmp[-1]
            print(f"Instance Run {instanceRun} Number {commitNumber} ...")

            #Read latency file for both types (old and new)
            microbenchs = load_microbench_csv(os.path.join(fullNumberFolder, "microbenchResults.csv"))
            microbenchs['instanceRun'] = instanceRun
            microbenchs['number'] = commitNumber
            
            aggr.extend(aggregate_data(microbenchs))


aggr = pd.concat(aggr)


Instance Run 1 Number 1 ...
  Found 57 benchmarks
  Errorcounter ([version1], [version2]): [[0, 0, 0], [0, 0, 0]]
Instance Run 1 Number 10 ...
  Found 57 benchmarks
  Errorcounter ([version1], [version2]): [[0, 0, 0], [0, 0, 0]]
Instance Run 1 Number 11 ...
  Found 57 benchmarks
  Errorcounter ([version1], [version2]): [[0, 0, 0], [0, 0, 0]]
Instance Run 1 Number 12 ...
  Found 57 benchmarks
  Errorcounter ([version1], [version2]): [[0, 0, 0], [0, 0, 0]]
Instance Run 1 Number 13 ...
  Found 57 benchmarks
  Errorcounter ([version1], [version2]): [[0, 0, 0], [0, 0, 0]]
Instance Run 1 Number 14 ...
  Found 57 benchmarks
  Errorcounter ([version1], [version2]): [[0, 0, 0], [0, 0, 0]]
Instance Run 1 Number 15 ...
  Found 57 benchmarks
  Errorcounter ([version1], [version2]): [[0, 0, 0], [0, 0, 0]]
Instance Run 1 Number 16 ...
  Found 57 benchmarks
  Errorcounter ([version1], [version2]): [[0, 0, 0], [0, 0, 0]]
Instance Run 1 Number 17 ...
  Found 57 benchmarks
  Errorcounter ([version1], [v

In [5]:
# Convert to data frame
df_detail = aggr
df_detail.sort_values(by=["name","number", "instanceRun","suiteRun"], inplace=True)
df_detail.describe()

,suiteRun,version,runtime,invocations
count,117920.000000,117920.000000,1.179200e+05,1.179200e+05
mean,1.000424,1.500000,5.366752e+06,1.929433e+05
std,0.816379,0.500002,1.290481e+07,3.856300e+05
min,0.000000,1.000000,6.864000e+02,1.600000e+01
25%,0.000000,1.000000,6.649000e+03,5.640000e+02
50%,1.000000,1.500000,4.145700e+04,2.894400e+04
75%,2.000000,2.000000,3.127707e+06,1.782435e+05
max,2.000000,2.000000,6.896352e+07,1.772119e+06


In [6]:
df_detail.to_csv(outputFile, sep=";")
